In [1]:
%matplotlib inline

import geopandas as gpd

import pandas as pd
import os
import requests
import folium
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# matplotlib에 “AppleGothc”(Mac) 또는 “Malgun Gothic”(Windows) 글꼴 적용

import matplotlib.font_manager as fm

for font in fm.fontManager.ttflist:
    if font.name in ['AppleGothic', 'Malgun Gothic']:
        plt.rcParams['font.family'] = font.name
        break

plt.rcParams['font.family']

['Malgun Gothic']

In [4]:
#2018년 7월 승차일 기준 1~4일의 버스 카드태깅 정보를 담고있습니다.
#미터!!!!!!!!!!!!!!!!!!!!!!!!!
TripChain = pd.read_csv('../Data/PJT001_TripChain.csv')

#17~18년 기준, 경기도 버스 정류장에 대한 정보를 담고있습니다.
# 성보님과 은솔님이 채워주신 데이터 바로 이용
StationTable = pd.read_csv('../Data/station_null.csv')

#2018년 7월 1~4일의 행정동별 이동 인구수 정보를 담고 있습니다.
AreaPeople = pd.read_csv('../Data/PJT001_sk_emd_od.csv')

#버스노선-정류장 매핑 테이블 정보 입니다.
RouteStationInfo = pd.read_csv('../Data/PJT001_routestationinfo.csv')

In [5]:
RouteStationmapp = pd.read_csv('../Data/PJT001_routestationmapping.csv')

In [7]:
TripChain.columns=['암호화카드번호', '트랜잭션ID', '환승횟수', '교통카드발행사ID',
       '총이용객수', '사용자구분', '교통수단CD1', '교통수단CD2',
       '교통수단CD3', '교통수단CD4', '교통수단CD5', '버스노선ID1',
       '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
       '차량ID1', '차량ID2', '차량ID3', '차량ID4',
       '차량ID5', '총통행거리', '총탑승시간', '총소요시간',
       '승차일시1', '승차일시2', '승차일시3', '승차일시4',
       '승차일시5', '하차일시1', '하차일시2', '하차일시3',
       '하차일시4', '하차일시5', '최초승차일시', '최종하차일시',
       '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
       '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3',
       '하차역ID4', '하차역ID5', '최초승차역ID', '최종하차역ID',
       '총이용금액', '수집건수', '트립체인완료코드']

In [29]:
TripChain['승차일시1'] = pd.to_datetime(TripChain['승차일시1'], format='%Y%m%d%H%M%S')
TripChain['승차일시2'] = pd.to_datetime(TripChain['승차일시2'], format='%Y%m%d%H%M%S')
TripChain['승차일시3'] = pd.to_datetime(TripChain['승차일시3'], format='%Y%m%d%H%M%S')
TripChain['승차일시4'] = pd.to_datetime(TripChain['승차일시4'], format='%Y%m%d%H%M%S')
TripChain['승차일시5'] = pd.to_datetime(TripChain['승차일시5'], format='%Y%m%d%H%M%S')
TripChain['하차일시1'] = pd.to_datetime(TripChain['하차일시1'], format='%Y%m%d%H%M%S')
TripChain['하차일시2'] = pd.to_datetime(TripChain['하차일시2'], format='%Y%m%d%H%M%S')
TripChain['하차일시3'] = pd.to_datetime(TripChain['하차일시3'], format='%Y%m%d%H%M%S')
TripChain['하차일시4'] = pd.to_datetime(TripChain['하차일시4'], format='%Y%m%d%H%M%S')
TripChain['하차일시5'] = pd.to_datetime(TripChain['하차일시5'], format='%Y%m%d%H%M%S')
TripChain['최초승차일시'] = pd.to_datetime(TripChain['최초승차일시'], format='%Y%m%d%H%M%S')

In [30]:
TripChain.drop_duplicates(subset =["암호화카드번호","최초승차일시"], 
                     keep = 'first', inplace = True) 

In [31]:
TripChain['날짜구분'] = TripChain['최초승차일시'].apply(lambda x: x.day)

In [34]:
TripChain_p_1 = TripChain[TripChain['사용자구분']==1]

일반 - 주말

In [35]:
TripChain_1_rest = TripChain_p_1[TripChain_p_1['날짜구분']==1]

In [68]:
p_1_hash_1 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for hour_1 in range(len(TripChain_1_rest)):
    tmp = TripChain_1_rest.iloc[hour_1]['최초승차일시'].hour
    p_1_hash_1[tmp] += 1
p_1_hash_1

{4: 329,
 5: 3207,
 6: 4452,
 7: 5641,
 8: 8854,
 9: 9876,
 10: 8683,
 11: 8167,
 12: 9604,
 13: 10053,
 14: 10550,
 15: 10498,
 16: 11245,
 17: 11770,
 18: 11649,
 19: 9884,
 20: 10232,
 21: 9860,
 22: 7736,
 23: 3197}

일반 - 주중

2일

In [88]:
TripChain_1_work = TripChain_p_1[TripChain_p_1['날짜구분']==2]
p_1_hash_1 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for hour_1 in range(len(TripChain_1_work)):
    tmp = TripChain_1_work.iloc[hour_1]['최초승차일시'].hour
    p_1_hash_1[tmp] += 1
p_1_hash_1

{4: 562,
 5: 6407,
 6: 16097,
 7: 30603,
 8: 29610,
 9: 19375,
 10: 13548,
 11: 13346,
 12: 14520,
 13: 14620,
 14: 14412,
 15: 16507,
 16: 18685,
 17: 23020,
 18: 30744,
 19: 19125,
 20: 16118,
 21: 15108,
 22: 11530,
 23: 4546}

3일

In [89]:
TripChain_1_work = TripChain_p_1[TripChain_p_1['날짜구분']==3]
p_1_hash_1 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for hour_1 in range(len(TripChain_1_work)):
    tmp = TripChain_1_work.iloc[hour_1]['최초승차일시'].hour
    p_1_hash_1[tmp] += 1
p_1_hash_1

{4: 656,
 5: 6982,
 6: 15960,
 7: 31680,
 8: 31193,
 9: 20532,
 10: 14460,
 11: 14398,
 12: 16452,
 13: 17160,
 14: 16117,
 15: 18306,
 16: 20789,
 17: 26459,
 18: 32741,
 19: 21899,
 20: 18676,
 21: 17957,
 22: 13830,
 23: 5883}

4일

In [90]:
TripChain_1_work = TripChain_p_1[TripChain_p_1['날짜구분']==4]
p_1_hash_1 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for hour_1 in range(len(TripChain_1_work)):
    tmp = TripChain_1_work.iloc[hour_1]['최초승차일시'].hour
    p_1_hash_1[tmp] += 1
p_1_hash_1

{4: 196,
 5: 1728,
 6: 3897,
 7: 7394,
 8: 7372,
 9: 4984,
 10: 3461,
 11: 3495,
 12: 3984,
 13: 4128,
 14: 3811,
 15: 4369,
 16: 5025,
 17: 6538,
 18: 7919,
 19: 5271,
 20: 4419,
 21: 4300,
 22: 3462,
 23: 1445}

어린이 - 주말

In [71]:
TripChain_p_2 = TripChain[TripChain['사용자구분']==2]

In [72]:
TripChain_2_rest = TripChain_p_2[TripChain_p_2['날짜구분']==1]
p_2_hash_1 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for hour_2 in range(len(TripChain_2_rest)):
    tmp = TripChain_2_rest.iloc[hour_2]['최초승차일시'].hour
    p_2_hash_1[tmp] += 1
p_2_hash_1

{4: 0,
 5: 3,
 6: 4,
 7: 7,
 8: 32,
 9: 42,
 10: 85,
 11: 79,
 12: 85,
 13: 131,
 14: 118,
 15: 103,
 16: 91,
 17: 100,
 18: 89,
 19: 43,
 20: 26,
 21: 14,
 22: 3,
 23: 1}

어린이 - 주중

2일

In [92]:
TripChain_2_work = TripChain_p_2[TripChain_p_2['날짜구분']==2]
p_2_hash_2 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for hour_2 in range(len(TripChain_2_work)):
    tmp = TripChain_2_work.iloc[hour_2]['최초승차일시'].hour
    p_2_hash_2[tmp] += 1
p_2_hash_2

{4: 0,
 5: 0,
 6: 3,
 7: 83,
 8: 332,
 9: 12,
 10: 15,
 11: 23,
 12: 37,
 13: 139,
 14: 143,
 15: 361,
 16: 340,
 17: 201,
 18: 152,
 19: 77,
 20: 48,
 21: 19,
 22: 14,
 23: 1}

3일

In [93]:
TripChain_2_work = TripChain_p_2[TripChain_p_2['날짜구분']==3]
p_2_hash_2 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for hour_2 in range(len(TripChain_2_work)):
    tmp = TripChain_2_work.iloc[hour_2]['최초승차일시'].hour
    p_2_hash_2[tmp] += 1
p_2_hash_2

{4: 0,
 5: 0,
 6: 3,
 7: 81,
 8: 368,
 9: 19,
 10: 10,
 11: 23,
 12: 55,
 13: 130,
 14: 184,
 15: 342,
 16: 397,
 17: 342,
 18: 224,
 19: 137,
 20: 89,
 21: 50,
 22: 22,
 23: 1}

4일

In [94]:
TripChain_2_work = TripChain_p_2[TripChain_p_2['날짜구분']==4]
p_2_hash_2 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for hour_2 in range(len(TripChain_2_work)):
    tmp = TripChain_2_work.iloc[hour_2]['최초승차일시'].hour
    p_2_hash_2[tmp] += 1
p_2_hash_2

{4: 0,
 5: 0,
 6: 2,
 7: 33,
 8: 94,
 9: 9,
 10: 6,
 11: 10,
 12: 41,
 13: 63,
 14: 74,
 15: 100,
 16: 99,
 17: 90,
 18: 70,
 19: 32,
 20: 24,
 21: 9,
 22: 2,
 23: 1}

청소년 - 주말

In [95]:
TripChain_p_4 = TripChain[TripChain['사용자구분']==4]

In [76]:
TripChain_4_rest = TripChain_p_4[TripChain_p_4['날짜구분']==1]
p_4_hash_1 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for hour_4 in range(len(TripChain_4_rest)):
    tmp = TripChain_4_rest.iloc[hour_4]['최초승차일시'].hour
    p_4_hash_1[tmp] += 1
p_4_hash_1

{4: 0,
 5: 35,
 6: 78,
 7: 168,
 8: 362,
 9: 588,
 10: 785,
 11: 868,
 12: 1041,
 13: 1111,
 14: 1043,
 15: 1081,
 16: 1091,
 17: 1094,
 18: 1054,
 19: 900,
 20: 816,
 21: 810,
 22: 842,
 23: 295}

청소년 - 주중

2일

In [96]:
TripChain_4_work = TripChain_p_4[TripChain_p_4['날짜구분']==2]
p_4_hash_2 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for hour_4 in range(len(TripChain_4_work)):
    tmp = TripChain_4_work.iloc[hour_4]['최초승차일시'].hour
    p_4_hash_2[tmp] += 1
p_4_hash_2

{4: 5,
 5: 37,
 6: 550,
 7: 3981,
 8: 5607,
 9: 317,
 10: 288,
 11: 763,
 12: 1085,
 13: 2042,
 14: 932,
 15: 1947,
 16: 2690,
 17: 6136,
 18: 1883,
 19: 1332,
 20: 1107,
 21: 1233,
 22: 1409,
 23: 346}

3일

In [97]:
TripChain_4_work = TripChain_p_4[TripChain_p_4['날짜구분']==3]
p_4_hash_2 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for hour_4 in range(len(TripChain_4_work)):
    tmp = TripChain_4_work.iloc[hour_4]['최초승차일시'].hour
    p_4_hash_2[tmp] += 1
p_4_hash_2

{4: 1,
 5: 52,
 6: 687,
 7: 4451,
 8: 5692,
 9: 287,
 10: 277,
 11: 1637,
 12: 2890,
 13: 4759,
 14: 1652,
 15: 1502,
 16: 2631,
 17: 4461,
 18: 1860,
 19: 1481,
 20: 1374,
 21: 1289,
 22: 1466,
 23: 427}

4일

In [98]:
TripChain_4_work = TripChain_p_4[TripChain_p_4['날짜구분']==4]
p_4_hash_2 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for hour_4 in range(len(TripChain_4_work)):
    tmp = TripChain_4_work.iloc[hour_4]['최초승차일시'].hour
    p_4_hash_2[tmp] += 1
p_4_hash_2

{4: 0,
 5: 15,
 6: 156,
 7: 1023,
 8: 1348,
 9: 87,
 10: 72,
 11: 511,
 12: 842,
 13: 1152,
 14: 477,
 15: 543,
 16: 556,
 17: 1020,
 18: 450,
 19: 373,
 20: 315,
 21: 327,
 22: 365,
 23: 110}

경로 - 주말

In [78]:
TripChain_p_8 = TripChain[TripChain['사용자구분']==8]

In [79]:
TripChain_8_rest = TripChain_p_8[TripChain_p_8['날짜구분']==1]
p_8_hash_1 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for hour_8 in range(len(TripChain_8_rest)):
    tmp = TripChain_8_rest.iloc[hour_8]['최초승차일시'].hour
    p_8_hash_1[tmp] += 1
p_8_hash_1

{4: 1,
 5: 22,
 6: 17,
 7: 12,
 8: 23,
 9: 31,
 10: 27,
 11: 20,
 12: 24,
 13: 23,
 14: 31,
 15: 31,
 16: 35,
 17: 38,
 18: 28,
 19: 20,
 20: 15,
 21: 8,
 22: 8,
 23: 3}

경로 - 주중

2일

In [99]:
TripChain_8_work = TripChain_p_8[TripChain_p_8['날짜구분']==2]
p_8_hash_2 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for hour_8 in range(len(TripChain_8_work)):
    tmp = TripChain_8_work.iloc[hour_8]['최초승차일시'].hour
    p_8_hash_2[tmp] += 1
p_8_hash_2

{4: 1,
 5: 26,
 6: 47,
 7: 53,
 8: 51,
 9: 54,
 10: 75,
 11: 68,
 12: 61,
 13: 59,
 14: 85,
 15: 66,
 16: 51,
 17: 41,
 18: 39,
 19: 30,
 20: 19,
 21: 14,
 22: 9,
 23: 4}

3일

In [100]:
TripChain_8_work = TripChain_p_8[TripChain_p_8['날짜구분']==3]
p_8_hash_2 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for hour_8 in range(len(TripChain_8_work)):
    tmp = TripChain_8_work.iloc[hour_8]['최초승차일시'].hour
    p_8_hash_2[tmp] += 1
p_8_hash_2

{4: 1,
 5: 27,
 6: 32,
 7: 67,
 8: 64,
 9: 61,
 10: 80,
 11: 63,
 12: 77,
 13: 81,
 14: 78,
 15: 85,
 16: 85,
 17: 55,
 18: 50,
 19: 44,
 20: 26,
 21: 25,
 22: 16,
 23: 3}

4일

In [101]:
TripChain_8_work = TripChain_p_8[TripChain_p_8['날짜구분']==4]
p_8_hash_2 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
for hour_8 in range(len(TripChain_8_work)):
    tmp = TripChain_8_work.iloc[hour_8]['최초승차일시'].hour
    p_8_hash_2[tmp] += 1
p_8_hash_2

{4: 1,
 5: 17,
 6: 12,
 7: 23,
 8: 20,
 9: 19,
 10: 24,
 11: 17,
 12: 23,
 13: 25,
 14: 22,
 15: 21,
 16: 15,
 17: 25,
 18: 23,
 19: 9,
 20: 11,
 21: 6,
 22: 5,
 23: 7}

In [82]:
all_t_1 = TripChain[TripChain['날짜구분']==1]
all_t_234 = TripChain[TripChain['날짜구분']!=1]

In [83]:
p_all_hash_1 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}
p_all_hash_2 = {4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0,
            11:0, 12:0, 13:0, 14:0, 15:0, 16:0, 17:0,
           18:0, 19:0, 20:0, 21:0, 22:0, 23:0}

In [84]:
for hour_ in range(len(all_t_1)):
    tmp = all_t_1.iloc[hour_]['최초승차일시'].hour
    p_all_hash_1[tmp] += 1

In [85]:
for hour_ in range(len(all_t_234)):
    tmp = all_t_234.iloc[hour_]['최초승차일시'].hour
    p_all_hash_2[tmp] += 1

In [86]:
p_all_hash_1

{4: 330,
 5: 3267,
 6: 4551,
 7: 5828,
 8: 9271,
 9: 10537,
 10: 9580,
 11: 9134,
 12: 10754,
 13: 11318,
 14: 11742,
 15: 11713,
 16: 12462,
 17: 13002,
 18: 12820,
 19: 10847,
 20: 11089,
 21: 10692,
 22: 8589,
 23: 3496}

In [87]:
p_all_hash_2

{4: 1423,
 5: 15291,
 6: 37446,
 7: 79472,
 8: 81751,
 9: 45756,
 10: 32316,
 11: 34354,
 12: 40067,
 13: 44358,
 14: 37987,
 15: 44149,
 16: 51363,
 17: 68388,
 18: 76155,
 19: 49810,
 20: 42226,
 21: 40337,
 22: 32130,
 23: 12774}

In [25]:
TripChain_p_4 = TripChain[TripChain['사용자구분']==4]
p_4_hash = {'4':0, '5':0, '6':0, '7':0, '8':0, '9':0, '10':0,
            '11':0, '12':0, '13':0, '14':0, '15':0, '16':0, '17':0,
           '18':0, '19':0, '20':0, '21':0, '22':0, '23':0}

for hour_4 in range(len(TripChain_p_4)):
    tmp = str(TripChain['최초승차일시'][hour_4].hour)
    p_4_hash[tmp] += 1

In [26]:
p_4_hash

{'4': 194,
 '5': 1770,
 '6': 2336,
 '7': 3141,
 '8': 4945,
 '9': 5601,
 '10': 5110,
 '11': 4877,
 '12': 5756,
 '13': 6052,
 '14': 6311,
 '15': 6224,
 '16': 6603,
 '17': 6830,
 '18': 6858,
 '19': 5744,
 '20': 5910,
 '21': 5690,
 '22': 4559,
 '23': 1859}

In [27]:
TripChain_p_8 = TripChain[TripChain['사용자구분']==8]
p_8_hash = {'4':0, '5':0, '6':0, '7':0, '8':0, '9':0, '10':0,
            '11':0, '12':0, '13':0, '14':0, '15':0, '16':0, '17':0,
           '18':0, '19':0, '20':0, '21':0, '22':0, '23':0}

for hour_8 in range(len(TripChain_p_8)):
    tmp = str(TripChain['최초승차일시'][hour_8].hour)
    p_8_hash[tmp] += 1

In [28]:
p_8_hash

{'4': 3,
 '5': 55,
 '6': 68,
 '7': 81,
 '8': 145,
 '9': 156,
 '10': 133,
 '11': 140,
 '12': 141,
 '13': 173,
 '14': 158,
 '15': 180,
 '16': 155,
 '17': 194,
 '18': 173,
 '19': 165,
 '20': 175,
 '21': 152,
 '22': 118,
 '23': 50}

In [102]:
hah = pd.read_csv('danbi_hwasung_routemapping.csv')

In [103]:
hah

,Unnamed: 0,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명,isHwasung,p_1,p_2,p_4,p_8
0,53,경기시내,경기고속,4100200,234000068,41002090,720-3,True,20430,68,899,0
1,2102,경기시내,(주)화성여객,4108600,233000133,41086002,116-3,True,11988,47,713,0
2,2114,경기시내,(주)화성여객,4108600,234001570,41086011,116-2,True,6677,40,580,0
3,2113,경기시내,(주)화성여객,4108600,234001568,41086008,721,True,5968,21,435,0
4,904,경기시내,경진여객,4100800,233000086,41008151,38,True,4403,18,165,0
5,2026,경기시내,화성운수,4107500,233000128,41075031,712,True,4194,16,144,0
6,2103,경기시내,(주)화성여객,4108600,233000135,41086005,8501,True,4069,3,91,0
7,2027,경기시내,화성운수,4107500,233000130,41075017,73-1,True,3474,25,355,0
8,1919,경기시내,제부여객,4106100,233000253,41061027,150,True,2862,13,54,0
9,2108,경기시내,(주)화성여객,4108600,233000254,41086013,203,True,2085,8,54,0
